In [12]:
%pylab inline
import logging as log
log.getLogger().setLevel(log.INFO)
import pandas as pd
from ipypb import track
import pandas as pd
from geopandas import GeoDataFrame
from pandas import DataFramed
from shapely.geometry import Point
import fiona
import regex
from datetime import datetime
from sharpy import *

import glob

names = ["RADARGRAM COLUMN", "TIME", "LATITUDE", "LONGITUDE", "MARS RADIUS", "SPACECRAFT RADIUS", 
                 "RADIAL VELOCITY", "TANGENTIAL VELOCITY", "SZA", "PHASE/1.0E16"]

types = [int, datetime, float, float, float, float, float, float, float, float]
names = [n.replace(" ", "_").replace("/", "_").lower() for n in names]


dtype = [t for t in zip(names, types)]

geom_f = "/data/SciBigData/radargrams/geom/"
pattern = geom_f + "/**/*_geom.lbl"
log.info(pattern)

allgeoms = glob.glob(pattern)

log.info(f"found {len(allgeoms)} radargrams geom-files")

DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
/usr/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f', 'log', 'dtype', 'datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"
INFO:root:/data/SciBigData/radargrams/geom//**/*_geom.lbl
INFO:root:found 21871 radargrams geom-files


Populating the interactive namespace from numpy and matplotlib


In [13]:
from shapely.geometry import Point, LineString, MultiLineString

In [15]:
# this is the actual loop injesting the geom files and simplyfying them
out = []
ids = []
for f in track(allgeoms):
    tab = lbl_to_geom(f)
    df = read_geom_lbl(tab)
    geometry = LineString([Point(xy) for xy in zip(df.longitude, df.latitude)])
    geometry= geometry.simplify(0.001)
    i = ids.append( match_sharad_id(f) )
    out.append(geometry)
    

 [████████████████████████████████████████████████████████████] 21871/21871 [30:31<00:00, 0.08s/it]

In [1]:
df = DataFrame()
df["track_id"] = ids

NameError: name 'DataFrame' is not defined

In [18]:
crs = fiona.crs.from_string("+proj=longlat +a=3396190 +b=3376200 +no_defs") # MARS 2000
gf = GeoDataFrame(df, crs=crs, geometry=out)

In [19]:
gf.to_file("alltracks")